In [190]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import torch 
import torch.nn as nn
from torch.utils.data import Dataset , DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score
import os 
import csv
import torch.functional as F 
from tqdm import tqdm

In [191]:
Labels_map = ["Fail","no Fail"]
device = torch.device("cuda")
BATCH_SIZE = 64
LR = 0.001
DROPOUT = 0.6
EPOCHS  = 100

In [192]:


class MachinesData(Dataset):
    def __init__(self,data,labels,device):
        self.data = torch.tensor(data,dtype=torch.float32,device=device)
        self.labels = torch.tensor(labels,dtype=torch.float32,device=device) 
        
    def __len__(self):
        return self.data.shape[0]
    def __getitem__(self, index) :
        return self.data[index],self.labels[index]


def load_data(path="Machines_2"):
    data = pd.read_csv(os.path.join(path,"data.csv"))

    labels = data.iloc[:,-1]
    data = data.iloc[:,:-1]

    normdata = (data- data.mean(axis=0))/data.std(axis=0)
    normdata = pd.concat([normdata,labels],axis=1)
    
    return normdata

def split_data(data,random_state = 12):
    train_data,test_data = train_test_split(data,test_size=0.2,random_state=random_state)
    return train_data,test_data


data = load_data()

train_data,test_data = split_data(data)
print(f"train_data :{train_data.shape[0]}")
print(f"test_data :{test_data.shape[0]}")


train_data :755
test_data :189


,footfall,tempMode,AQ,USS,CS,VOC,RP,IP,Temperature,fail
224,-0.125975,-0.645350,-0.226087,0.766324,-1.098254,-1.250215,0.362694,-0.978985,-0.557605,1
761,-0.033605,-0.271831,-0.921287,-0.679050,1.265162,-1.250215,0.058245,-0.978985,0.781356,0
339,-0.278385,-0.271831,-0.226087,0.043637,0.477357,-1.250215,-0.246204,-0.353706,-0.222865,0
42,-0.181397,-1.392390,-1.616486,0.043637,0.477357,-0.810333,-1.524888,0.896851,-2.231307,0
579,-0.273766,-1.018870,-1.616486,0.043637,1.265162,-1.250215,-0.915991,0.271572,0.613986,0


In [193]:
train_dataset = MachinesData(train_data.iloc[:,:-1].values,train_data.iloc[:,-1].values,device)
test_dataset = MachinesData(test_data.iloc[:,:-1].values,test_data.iloc[:,-1].values,device)

train_loader = DataLoader(train_dataset,batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset,batch_size=BATCH_SIZE)

In [194]:
class FailureWatch(torch.nn.Module):
    def __init__(self,num_feat,num_classes,dropout_rate):
        super(FailureWatch,self).__init__()

        self.drop = nn.Dropout(dropout_rate)
        self.act = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

        self.layer1 = nn.Linear(num_feat,128)
        self.layer2 = nn.Linear(128,256)
        self.layer3 = nn.Linear(256,1024)
        self.layer4 = nn.Linear(1024,1024)
        self.out_layer = nn.Linear(1024,num_classes)
    
    def forward(self,x):
        x = self.drop(self.act(self.layer1(x)))
        x = self.drop(self.act(self.layer2(x)))
        x = self.drop(self.act(self.layer3(x)))
        x = self.drop(self.act(self.layer4(x)))
        x = self.out_layer(x)
        return self.sigmoid(x)

num_features = train_data.shape[1]-1
num_classes = 1
model = FailureWatch(num_features,num_classes,DROPOUT).to(device)

In [195]:
def evaluate(model:FailureWatch,loader):
    model.eval()
    preds = []
    real = []

    with torch.no_grad() :
        for x , y in loader :
            out = model(x)
            out = out.to("cpu").numpy()
            y = y.to("cpu").numpy()

            pred = [0 if i<0.5 else 1 for i in out ]
            preds +=pred
            real += list(y)
    
    f1 = f1_score(preds,real)
    acc = accuracy_score(preds,real)
    print(f"accuracy score :{acc:.3f} ,f1 score :{f1:.3f}")
    model.train()
    return acc


def train(num_epochs,model:FailureWatch,train_loader):

    optimizer = torch.optim.Adam(model.parameters(),LR)
    loss_function = nn.BCELoss()
    
    for epoch in range(num_epochs):
        pbar = tqdm(total=train_data.shape[0])
        pbar.set_description(f"Epoch {epoch}:")
        for batch_idx,(batch_data,batch_labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(batch_data)
            loss = loss_function(outputs.view(-1),batch_labels)
            loss.backward()
            optimizer.step()
            pbar.update(BATCH_SIZE)

        pbar.close()
        
        print(f"Epoch {epoch}: loss={float(loss):.4f}  ,",end=" ")
        evaluate(model,train_loader)

    return model





In [196]:
model = train(EPOCHS,model,train_loader)





Epoch 0:: : 768it [00:00, 8954.29it/s]           


Epoch 0: loss=0.4861  , accuracy score :0.870 ,f1 score :0.829






Epoch 1:: : 768it [00:00, 20249.35it/s]          


Epoch 1: loss=0.2322  , accuracy score :0.901 ,f1 score :0.884






Epoch 2:: : 768it [00:00, 19239.93it/s]          


...

Epoch 97: loss=0.1468  , accuracy score :0.938 ,f1 score :0.927






Epoch 98:: : 768it [00:00, 19236.94it/s]          


Epoch 98: loss=0.1454  , accuracy score :0.940 ,f1 score :0.930






Epoch 99:: : 768it [00:00, 18782.33it/s]          

Epoch 99: loss=0.1071  , accuracy score :0.942 ,f1 score :0.932


In [197]:
acc = evaluate(model,test_loader)

accuracy score :0.926 ,f1 score :0.908


In [198]:
torch.save(model.state_dict(),f"model/acc_{acc}_weights.pt")